## Automating Bicycle Sales Reporting for Informed Decision-Making
Regularly reporting on vital business metrics is a fundamental duty for analysts, one that holds immense value for the team. However, the traditional method of manually compiling these reports often proves time-consuming and prone to errors.

By automating the reporting process, we streamline operations and ensure that decision-makers have access to up-to-date insights without delay. This not only improves efficiency but also enhances the accuracy and reliability of our reporting, ultimately leading to better-informed decisions.

With the combination of SQL for data retrieval and Python for data analysis and visualization, we create a seamless workflow that transforms raw data into actionable insights. Visualizations such as charts and graphs help to distill complex data into easily understandable formats, making it accessible to stakeholders across the organization.

Additionally, DataLab’s scheduling capabilities allow us to set up automated tasks for regular reporting, ensuring that the insights are delivered on time without manual intervention. This not only saves time but also reduces the likelihood of human errors, thereby increasing the confidence in the reported metrics.

Overall, by adopting this automated reporting approach, we enhance efficiency, accuracy, and reliability in reporting on key business metrics, empowering stakeholders to make informed decisions based on timely and insightful data analysis.

In this project, I'll showcase this workflow by reporting on bicycle sales and inventory data stored in Microsoft SQL Server data. Leveraging SQL, Python, and DataLab’s scheduling capabilities, we can automatically report on business metrics with insightful visualizations, saving time and reducing errors in the process.

In [6]:
# Install and load necessary packages
!pip install slack_sdk
from slack_sdk import WebClient
import os

Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Time of the last scheduled run
from datetime import datetime
print(datetime.now())

2024-04-24 22:48:01.430939


## Total sales by month

In [8]:
-- Get total sales over time
SELECT 
      DATEFROMPARTS(DATEPART(year, o.order_date), DATEPART(month, o.order_date),1) AS month,
	  SUM(quantity * list_price * (1-discount)) AS total_sales
FROM sales.orders AS o
INNER JOIN sales.order_items AS oi ON o.order_id = oi.order_id
WHERE o.order_date < '2018-05-01'
GROUP BY DATEFROMPARTS(DATEPART(year, o.order_date), DATEPART(month, o.order_date),1)
ORDER BY month;

,month,total_sales
0,2016-01-01 00:00:00+00:00,215146.4241
1,2016-02-01 00:00:00+00:00,156112.3228
2,2016-03-01 00:00:00+00:00,180600.3285
3,2016-04-01 00:00:00+00:00,167144.0512
4,2016-05-01 00:00:00+00:00,205270.0091
5,2016-06-01 00:00:00+00:00,210562.1245
6,2016-07-01 00:00:00+00:00,199556.8089
7,2016-08-01 00:00:00+00:00,225657.3767
8,2016-09-01 00:00:00+00:00,273091.6097
9,2016-10-01 00:00:00+00:00,212078.0805


In [9]:
import plotly.express as px

# Plotting total sales over time using Plotly
fig = px.line(sales_over_time, x='month', y='total_sales', title='Total Sales Over Time', markers=True)
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales', xaxis_tickangle=-45)
fig.show()

## Top 3 best sellers

In [14]:
-- Get top three of best sellers in March 2018
SELECT TOP 3
      CONCAT(first_name, ' ', last_name) AS full_name, 
	  SUM(quantity * list_price * (1-discount)) AS sales_per_staff
FROM sales.orders AS o
INNER JOIN sales.order_items AS oi ON o.order_id = oi.order_id
INNER JOIN sales.staffs AS s ON o.staff_id = s.staff_id
WHERE DATEPART(year, o.order_date) = 2018
      AND DATEPART(month, o.order_date) = 4
GROUP BY CONCAT(first_name, ' ', last_name)
ORDER BY sales_per_staff DESC;

,full_name,sales_per_staff
0,Venita Daniel,275594.3638
1,Genna Serrano,174986.5221
2,Marcelene Boyer,161670.5474


## Scheduling Automated Runs
Once I've completed and thoroughly tested my Jupyter Notebook, the next step is to schedule it for automatic execution at regular intervals. This can be achieved using external tools or services such as 'cron' on Unix-like systems or Task Scheduler on Windows. However, for this project, I've chosen to leverage Workspace from DataCamp, which offers a built-in capability similar to Task Scheduler, akin to 'cron' on Unix-like systems.

I'll configure the scheduler within Workspace to execute the notebook automatically on a weekly basis. This ensures that the analysis is consistently updated with fresh data, eliminating the need for manual intervention.

![Screenshot 2024-04-24 at 3.33.34 PM](Screenshot%202024-04-24%20at%203.33.34%20PM.png)


## Creating a Slack app from scratch
To efficiently report our analysis findings to stakeholders via Slack, I will develop a new application from scratch on the Slack platform, accessible through https://api.slack.com/apps. This application will be named "Slack Reporter."

To accomplish this, we must:

1.- Grant the necessary scopes to our bot.

2.- Install the OAuth Tokens into our workspace.

![Screenshot 2024-04-24 at 4.19.38 PM](Screenshot%202024-04-24%20at%204.19.38%20PM.png)


![Screenshot 2024-04-24 at 4.24.41 PM](Screenshot%202024-04-24%20at%204.24.41%20PM.png)


Next, we will obtain our Bot User OAuth Token. For security purposes, I will store the OAuth Token as an environment variable within the workspace.

![Screenshot 2024-04-24 at 5.41.24 PM](Screenshot%202024-04-24%20at%205.41.24%20PM.png)


To achieve this, I will navigate to the "OAuth & Permissions" section in the left menu, select "Add Environment Variables," and input the required information, including the OAuth Token.

![Screenshot 2024-04-24 at 5.42.21 PM](Screenshot%202024-04-24%20at%205.42.21%20PM.png)


![Screenshot 2024-04-24 at 4.34.50 PM](Screenshot%202024-04-24%20at%204.34.50%20PM.png)


Once the OAuth Token is stored as an environment variable, we will proceed to add our new application on Slack.

![Screenshot 2024-04-24 at 4.45.00 PM](Screenshot%202024-04-24%20at%204.45.00%20PM.png)


## Report metrics to Slack
The following code retrieves information about the latest sales numbers and top sellers, formats this information into a message, and then posts the message to a designated Slack channel. Let's break it down:

1.- The code begins by importing the format_currency function from the babel.numbers module. This function is used to format currency values in a specific locale.

2.- The variable last_sales_number is assigned the value of the total sales from the latest month in the sales_over_time DataFrame. This is achieved by filtering the DataFrame to get the row with the maximum month value and then extracting the total sales value from that row.

3.- The variable sellers_list is created by applying a lambda function to each row of the top_sellers DataFrame. This lambda function formats each row into a string containing the seller's full name and their sales per staff, formatted as currency using the format_currency function.

4.- The variable sellers_overview is created by joining the elements of the sellers_list with newline characters ("\n"). This results in a single string containing each seller's information on a separate line.

5.- The variable text is formatted using an f-string literal. It contains placeholders for the last month's sales amount ({format_currency(last_sales_number, 'USD')}) and the sellers' overview ({sellers_overview}). Additionally, there's a link to my workspace provided at the end of the message.

6.- The code uses the WebClient class from the slack module to post the message (text) to a specific Slack channel (sales-tracking). It uses the OAuth token stored in the SLACK_OAUTH_TOKEN environment variable to authenticate with the Slack API.

In [28]:
from babel.numbers import format_currency

last_sales_number = sales_over_time[sales_over_time['month'] == sales_over_time['month'].max()]['total_sales'].values[0]

sellers_list = top_sellers \
      .apply(lambda x: f"• {x['full_name']} ({format_currency(x['sales_per_staff'], 'USD')})", axis="columns")
sellers_overview = "\n".join(sellers_list)

text = f"""The past month we did {format_currency(last_sales_number, 'USD')} in sales!

Our sales all stars:
{sellers_overview}

_<https://www.datacamp.com/datalab/w/0a02369c-1890-4087-8658-74766b8519d2/edit/automating_bicycle_sales_reporting.ipynb|Link to workspace>_
"""

client = WebClient(token = os.environ["SLACK_OAUTH_TOKEN"])
client.chat_postMessage(channel = "sales-tracking", text = text)

## Et voilà! 
We now have a recurring message system capable of reporting our analysis findings to stakeholders through Slack on a monthly, weekly, or daily basis. For instance, in this project, our bot provides monthly sales figures along with a list of the top three sellers. Please note that there is a link provided at the bottom of the bot message, allowing anyone to access the analysis and delve deeper into their questions.

![Screenshot 2024-04-24 at 5.26.56 PM](Screenshot%202024-04-24%20at%205.26.56%20PM.png)
